In [ ]:
import json
from fhir.resources.questionnaireresponse import QuestionnaireResponse
from fhir.resources.questionnaire import Questionnaire
from fhir.resources.patient import Patient
from fhir.resources.humanname import HumanName

def json_to_fhir(questionnaire_file, response_file):
    """Converts JSON files into FHIR resources."""
    with open(questionnaire_file, 'r') as file:
        questionnaire_data = json.load(file)
    questionnaire = Questionnaire.parse_obj(questionnaire_data)
    
    with open(response_file, 'r') as file:
        response_data = json.load(file)
    response = QuestionnaireResponse.parse_obj(response_data)

    return questionnaire, response

def initialize_patient():
    """Initializes and returns a new Patient resource."""
    pat = Patient()
    pat.name = [HumanName(use='official')]
    return pat

def apply_mappings(questionnaire, response, patient):
    """Applies mappings from the questionnaire to the patient based on responses."""
    # Mapping from Questionnaire codes to Patient attributes
    code_to_patient_attr = {
        "first-name": lambda value: setattr(patient.name[0], 'given', [value]),
        "lastname": lambda value: setattr(patient.name[0], 'family', value),
        "sexatbirth": lambda value: setattr(patient, 'gender', value),
        "dob": lambda value: setattr(patient, 'birthDate', value)
    }

    # Create a lookup for linkId to codes
    linkid_to_code = {
        item.linkId: next((code.code for code in item.code), None) for item in questionnaire.item
    }

    # Map responses to Patient fields
    for item in response.item:
        code = linkid_to_code.get(item.linkId)
        if code in code_to_patient_attr:
            value = (item.answer[0].valueString if hasattr(item.answer[0], 'valueString') else
                     item.answer[0].valueCoding.display if hasattr(item.answer[0], 'valueCoding') else
                     item.answer[0].valueDate if hasattr(item.answer[0], 'valueDate') else None)
            print(f"Processing {code} with value {value}")  # Debug output
            if value:
                code_to_patient_attr[code](value)

def main():
    questionnaire_file = 'q.json'
    response_file = 'qr.json'
    
    # Convert JSON files to FHIR resources
    questionnaire, response = json_to_fhir(questionnaire_file, response_file)
    
    # Initialize the patient resource
    patient = initialize_patient()
    
    # Apply mappings to populate the patient resource
    apply_mappings(questionnaire, response, patient)
    
    # Output the populated patient resource
    print(patient.json(indent=4))

if __name__ == "__main__":
    main()


# Current output from script above using a q and qr

Processing first-name with value Bart
Processing lastname with value Simpson
Processing sexatbirth with value None
Processing dob with value None
{
    "resourceType": "Patient",
    "name": [
        {
            "use": "official",
            "family": "Simpson",
            "given": [
                "Bart"
            ]
        }
    ]
}